In [61]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20
from os.path import dirname, join

import numpy as np
import pandas.io.sql as psql
import sqlite3 as sql

from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, Div
from bokeh.models.widgets import Slider, Select, TextInput, DateRangeSlider, DateSlider, MultiSelect, Button, Toggle
from bokeh.io import curdoc
from datetime import date, timedelta
from bokeh.models import HoverTool

output_notebook()

Loading BokehJS ...

# Preprocessing

In [51]:
df = pd.read_csv("AllBirdsv4.csv")
df_test = pd.read_csv("Test Birds Location.csv")

# replace "?" characters
#df = df.replace('?', None)
df["Y"] = df["Y"].map(lambda y: y.replace("?",""))
df["Y"] = df["Y"].astype('int64')
        
# Add a specific color for each specie according to Category20
birds_types = df['English_name'].unique()
n_birds_types = len(birds_types)
birds_types_colors = pd.Series(Category20[n_birds_types], index = birds_types).rename('color')
df = df.join(birds_types_colors, on = "English_name")        
df['alpha'] = 1

# Convert date string to datetime
a = pd.to_datetime(df['Date'], errors = 'coerce', format = "%m/%d/%Y").dropna()
b = pd.to_datetime(df['Date'], errors = 'coerce', format = "%Y-%m-00").dropna()
df['T'] = pd.concat([a,b], axis = 0)

df.head()

,File ID,English_name,Vocalization_type,Quality,Time,Date,X,Y,color,alpha,T
0,402254,Rose-crested Blue Pipit,call,no score,13:30,2/8/2018,49,63,#1f77b4,1,2018-02-08
1,406171,Rose-crested Blue Pipit,call,A,7:48,6/7/2017,125,133,#1f77b4,1,2017-06-07
2,405901,Rose-crested Blue Pipit,call,A,12:00,2/8/2018,58,76,#1f77b4,1,2018-02-08
3,405548,Rose-crested Blue Pipit,song,A,11:00,3/10/2018,55,125,#1f77b4,1,2018-03-10
4,401782,Rose-crested Blue Pipit,song,A,6:00,6/29/2008,129,123,#1f77b4,1,2008-06-29


# Birdmap

In [71]:
def bird_map(doc):
    desc = Div(text="Yolo bonjour", width=800)

    # Create Input controls
    date_input = DateSlider(title="Date d'observation", start=df['T'].min(), end=df['T'].max(),
                            value=date(2017, 9, 7), step=1, format = "%b %Y")


    bird_types_input = MultiSelect(title="Espèces", value=["All"],
                   options= ['All'] + list(birds_types) , size = n_birds_types + 1)
    
    button = Toggle(label="Foo", button_type="success")
    
    
    
    # Create Column Data Source that will be used by the plot
    source = ColumnDataSource(data=dict(x=[], y=[], color=[], alpha=[], name = []))

    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

    p = figure(title="Map with birds through time", tools = TOOLS, x_range=(0, 200), y_range=(0, 200))
    # plot map
    p.image_url( url=["map.jpg"],
             x=0, y=0, w=200, h=200, anchor="bottom_left")
    p.circle(x="x", y="y", source=source, size=12, color="color", line_color=None, fill_alpha="alpha")
    
    
    hover = HoverTool()
    hover.tooltips = [
        ("Bird type", "@name"),
        ("Position", "(@x, @y)"),
    ]
    p.tools = [hover]



    def select_obs():
        bird_types = bird_types_input.value
        if "All" not in bird_types:
            selector_types = df['English_name'].isin(bird_types)
        else:
            selector_types = [True] * df.shape[0]

        a = date_input.value - timedelta(days=120)
        b = date_input.value + timedelta(days=120)
        selector_date = ((a < df['T']) & (df['T'] < b))

        return df.loc[selector_date & selector_types] # Selection

    def update():
        df_selected = select_obs()       

        source.data = dict(
            x=df_selected['X'],
            y=df_selected['Y'],
            color=df_selected["color"],
            alpha=df_selected["alpha"],
            name=df_selected["English_name"],
        )

    controls = [date_input, bird_types_input]
    for control in controls:
        control.on_change('value', lambda attr, old, new: update())
    controls += [button]

    sizing_mode = 'fixed'  # 'scale_width' also looks nice with this example

    inputs = widgetbox(*controls, sizing_mode=sizing_mode)
    l = layout([
        [desc],
        [inputs, p],
    ], sizing_mode=sizing_mode)

    update()  # initial load of the data

    doc.add_root(l)

    
show(bird_map)